In [16]:
import pandas as pd
import nltk
import re
import os
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

In [15]:
#load sample file
example_file = "./tweets/J.P. Morgan.csv"
tweets_df = pd.read_csv(example_file, index_col=0)
print(tweets_df.iloc[0,0])
tweets_df

“The vaccine news and the focus on that is the most important thing for the market at the moment,” said Stephanie R… https://t.co/G8PdiRLkVt


,0
2020-12-08 23:51:00,“The vaccine news and the focus on that is the...
2020-12-08 23:30:03,The J.P. Morgan Healthcare #Conference is goin...
2020-12-08 23:17:45,RT @NathaniNoShame: every night in fear that s...
2020-12-08 23:10:42,Alphabet Inc. (GOOG) surprised the market with...
2020-12-08 23:07:06,@TheSolariReport Millionaires don't use Astrol...
...,...
2020-11-30 00:41:31,The Liquidating Team in play.\nFirst the manip...
2020-11-30 00:26:19,"J P Morgan, Ghost of Rich Dude's Past"
2020-11-29 23:42:38,"Go as far as you can see; when you get there, ..."
2020-11-29 23:05:40,"Had a great morning at BWW, with my son eating..."


In [ ]:
#remove stopword
#remove punctuations
#remove special characters
#do we need to stem the words? 
#tokenize

In [127]:
emoji_pattern = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"  # emoticons
    u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
    u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
    u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
    u"(\ud83c[\udde0-\uddff])"  # flags (iOS)
    "+", flags=re.UNICODE)

def remove_emoji(text):
    return emoji_pattern.sub(r'', text)

def preprocess(sentence):
    special_marks = ['&', '\n', '@', '$', '#', '☆', '!', '(', ')'
                     ',', '.', ';', ':', '💰', '✅']
    
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence = re.sub('@[^, ]*', '', sentence) #remove twitter user name after @
    for sm in special_marks:
            sentence = sentence.replace(sm, "")
            
    sentence = sentence.replace('{html}',"") 
    sentence = remove_emoji(sentence)
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    #stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in filtered_words]
    #return " ".join(filtered_words)
    return lemma_words

def preprocess_tweet(file):
    tweets_df = pd.read_csv(file, index_col=0)
    # drop duplicate
    tweets_df.drop_duplicates(inplace=True)
    
    tweets_df['cleanText']=tweets_df['0'].map(lambda s:preprocess(s)) 
    
    return tweets_df

tweets_clean = preprocess_tweet(example_file)

In [204]:
tweets_clean

,0,cleanText,pos_count,word_count,neg_count
2020-12-08 23:51:00,“The vaccine news and the focus on that is the...,"[vaccine, news, focus, important, thing, marke...",1,9,0
2020-12-08 23:30:03,The J.P. Morgan Healthcare #Conference is goin...,"[morgan, healthcare, conference, going, virtua...",0,8,0
2020-12-08 23:17:45,RT @NathaniNoShame: every night in fear that s...,"[every, night, fear, someone, knew, would, kil...",0,12,2
2020-12-08 23:10:42,Alphabet Inc. (GOOG) surprised the market with...,"[alphabet, inc, goog, surprised, market, resul...",0,12,0
2020-12-08 23:07:06,@TheSolariReport Millionaires don't use Astrol...,"[millionaire, use, astrology, billionaire, doj...",0,6,0
...,...,...,...,...,...
2020-11-30 00:48:47,@John11019 he was deceived by J.P. Morgan and ...,"[deceived, morgan, thomas, edison]",0,4,0
2020-11-30 00:41:31,The Liquidating Team in play.\nFirst the manip...,"[liquidating, team, playfirst, manipulation, e...",0,9,2
2020-11-30 00:26:19,"J P Morgan, Ghost of Rich Dude's Past","[morgan, ghost, rich, dude, past]",1,5,0
2020-11-29 23:05:40,"Had a great morning at BWW, with my son eating...","[great, morning, bww, son, eating, wing, parki...",1,12,0


## Get positive & Negative word count

In [205]:
pos_list = list(pd.read_csv('./opinion-lexicon-English/positive-words.txt',header = None)[0])
neg_list = list(pd.read_csv('./opinion-lexicon-English/negative-words.txt',header = None,encoding='latin-1')[0])

In [211]:
#record positive and negative word counts

def count_words(tweets_clean):
    #count the number of positive and negative words. Read in dataframe with column['cleanText']
    pos_count_list = []
    neg_count_list = []

    for index, row in tweets_clean.iterrows(): 
        pos_count = 0
        neg_count = 0 
        for x in row['cleanText']:
            if x in pos_list:
                pos_count += 1
            if x in neg_list:
                neg_count += 1
        pos_count_list.append(pos_count)
        neg_count_list.append(neg_count)


    tweets_clean['pos_count'] = pos_count_list
    tweets_clean['neg_count'] = neg_count_list
    tweets_clean['word_count'] = [len(x) for x in tweets_clean.cleanText]
    tweets_clean['pct_pos'] = tweets_clean['pos_count']/tweets_clean['word_count']
    tweets_clean['pct_neg'] = tweets_clean['neg_count']/tweets_clean['word_count']
    return tweets_clean

tweets_word_count = count_words(tweets_clean)

In [212]:
tweets_word_count

,0,cleanText,pos_count,word_count,neg_count,pct_pos,pct_neg
2020-12-08 23:51:00,“The vaccine news and the focus on that is the...,"[vaccine, news, focus, important, thing, marke...",1,9,0,0.111111,0.000000
2020-12-08 23:30:03,The J.P. Morgan Healthcare #Conference is goin...,"[morgan, healthcare, conference, going, virtua...",0,8,0,0.000000,0.000000
2020-12-08 23:17:45,RT @NathaniNoShame: every night in fear that s...,"[every, night, fear, someone, knew, would, kil...",0,12,2,0.000000,0.166667
2020-12-08 23:10:42,Alphabet Inc. (GOOG) surprised the market with...,"[alphabet, inc, goog, surprised, market, resul...",0,12,0,0.000000,0.000000
2020-12-08 23:07:06,@TheSolariReport Millionaires don't use Astrol...,"[millionaire, use, astrology, billionaire, doj...",0,6,0,0.000000,0.000000
...,...,...,...,...,...,...,...
2020-11-30 00:48:47,@John11019 he was deceived by J.P. Morgan and ...,"[deceived, morgan, thomas, edison]",0,4,0,0.000000,0.000000
2020-11-30 00:41:31,The Liquidating Team in play.\nFirst the manip...,"[liquidating, team, playfirst, manipulation, e...",0,9,2,0.000000,0.222222
2020-11-30 00:26:19,"J P Morgan, Ghost of Rich Dude's Past","[morgan, ghost, rich, dude, past]",1,5,0,0.200000,0.000000
2020-11-29 23:05:40,"Had a great morning at BWW, with my son eating...","[great, morning, bww, son, eating, wing, parki...",1,12,0,0.083333,0.000000
